# Deploy Model

## Get workspace

In [2]:
import sys, os

import azureml.core
print(azureml.core.VERSION)

1.0.8


In [3]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Found the config file in: /datadrive01/amlSDKAzureChestXray/code/aml_config/config.json
sdk-chest-xray
ghiordanchestxray03rsg
eastus
edf507a2-6235-46c5-b560-fd463ba2e771


## Organize Paths

In [4]:
import azure_chestxray_utils

In [5]:
base_dir = './models'

In [12]:
prj_consts = azure_chestxray_utils.chestxray_consts()

data_base_output_dir=os.path.join(base_dir, 
                                  os.path.join(*(prj_consts.BASE_OUTPUT_DIR_list)))

weights_dir = os.path.join(data_base_output_dir, os.path.join(*(prj_consts.MODEL_WEIGHTS_DIR_list))) 
fully_trained_weights_dir = os.path.join(data_base_output_dir, os.path.join(*(prj_consts.FULLY_PRETRAINED_MODEL_DIR_list))) 

model_file_name= 'azure_chest_xray_14_weights_712split_epoch_250_val_loss_179-4776.hdf5'

# model_file_name = 'azure_chest_xray_14_weights_712split_epoch_300_val_loss_361.7687.hdf5'

## Register Model

In [13]:
#Register the model
from azureml.core.model import Model

if len(Model.list(ws, tags={'project': 'chest-xray', 'val_loss' : '179'})) != 0:
    for m in Model.list(ws, tags={'project': 'chest-xray'}):
    # for m in r_models:
        print("Name:", m.name,"\tVersion:", m.version, "\tDescription:", m.description, m.tags)
    model = Model.list(ws, tags={'project': 'chest-xray'})[0]
    print(model.name, model.description, model.version, sep = '\t')
else:
    model = Model.register(model_path = model_file_name, # this points to a local file
                           model_name = "chest_xray_model_179.hdf5", # this is the name the model is registered as
                           tags = {'area': "health", 'type': "classification", 'project': 'chest-xray', 'val_loss' : '179'},
                           description = "Predict diseases with chest x-ray image classification",
                           workspace = ws)

print(model.name, model.description, model.version)

Registering model chest_xray_model_179.hdf5
chest_xray_model_179.hdf5 Predict diseases with chest x-ray image classification 1


## Create Image

In [5]:
%%writefile score.py

from azureml.core.model import Model
import os, sys, pickle, base64
from keras.layers import Dense
import keras_contrib
from keras_contrib.applications.densenet import DenseNetImageNet121
import numpy as np
import json
import pandas as pd
import azure_chestxray_utils, azure_chestxray_keras_utils, azure_chestxray_cam

#Parameters 
global as_string_b64encoded_pickled_data_column_name
as_string_b64encoded_pickled_data_column_name   = 'encoded_image'

def init():
    global model
    model_path = Model.get_model_path("chest_xray_model_179.hdf5")
    model = azure_chestxray_keras_utils.build_model(keras_contrib.applications.densenet.DenseNetImageNet121)
    model.load_weights(model_path)
    
def run(data, with_cam = False): 
    input_cv2_image = np.array(json.loads(data)['data'])
    predictions, serialized_cam_image, predicted_disease_index, direct_result = get_image_score_and_serialized_cam(input_cv2_image)

    outDict = {"predictedDiseaseIndex": int(predicted_disease_index), "chestXrayScore": str(predictions)}
    
    if with_cam:
        outDict["chestXrayCAM"]= as_string_b64encoded_pickled(serialized_cam_image)
        
    return json.dumps(outDict)
        

####################################
# Utils
####################################
def as_string_b64encoded_pickled(input_object):
     #b64encode returns bytes class, make it string by calling .decode('utf-8')
    return (base64.b64encode(pickle.dumps(input_object))).decode('utf-8')

def unpickled_b64decoded_as_bytes(input_object):
    if input_object.startswith('b\''):
        input_object = input_object[2:-1]
    # make string bytes
    input_object   =  input_object.encode('utf-8')
    #decode and the unpickle the bytes to recover original object
    return (pickle.loads(base64.b64decode(input_object)))

def get_image_score_and_serialized_cam(crt_cv2_image):
    prj_consts = azure_chestxray_utils.chestxray_consts()
    crt_cv2_image = azure_chestxray_utils.normalize_nd_array(crt_cv2_image)
    crt_cv2_image = 255*crt_cv2_image
    direct_result = model.predict(np.expand_dims(crt_cv2_image,0))
    crt_cv2_image=crt_cv2_image.astype('uint8')
    predictions, cam_image, predicted_disease_index = \
    azure_chestxray_cam.get_score_and_cam_picture(crt_cv2_image, model)
    blended_image = azure_chestxray_cam.process_cam_image(cam_image, crt_cv2_image)
    serialized_image = azure_chestxray_cam.plot_cam_results(blended_image, cam_image, crt_cv2_image, \
                 prj_consts.DISEASE_list[predicted_disease_index])
    return predictions, serialized_image, predicted_disease_index, direct_result

#for testing only
def test_image():
    resized_height = 224
    resized_width = 224
    
    #init local model
    global model
    model = azure_chestxray_keras_utils.build_model(keras_contrib.applications.densenet.DenseNetImageNet121)
    model.load_weights('azure_chest_xray_14_weights_712split_epoch_250_val_loss_179-4776.hdf5')
    
    #script for later use
    import cv2
    image_dir = "./../../../data/chestxray/ChestX-ray8/ChestXray-NIHCC/images"
    image_name = "00000003_000.png"
    full_path = os.path.join(image_dir, image_name)
    cv2_image = cv2.resize(cv2.imread(full_path), (resized_height, resized_width))
    
#     ans = model.predict(np.expand_dims(cv2_image,0))
    
    test_images = json.dumps({"data": cv2_image.tolist()})
    test_images = bytes(test_images, encoding = "utf8")
    ans = run(test_images, True)
    
    return ans


Overwriting score.py


In [97]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(pip_packages= ['azureml-defaults', 'tensorflow','tensorflow-tensorboard','numpy==1.14.5', 'git+git://github.com/keras-team/keras.git',
                                                'git+https://www.github.com/keras-team/keras-contrib.git',
                                                'http://download.pytorch.org/whl/cu90/torch-0.4.0-cp36-cp36m-linux_x86_64.whl'], 
                                 conda_packages=['h5py', 'notebook=5.6.0', 'opencv', 'tqdm', 'matplotlib', 'pandas'])

# 'git+git://github.com/keras-team/keras.git',
#                                                 'git+https://www.github.com/keras-team/keras-contrib.git',
#                                                 'http://download.pytorch.org/whl/cu90/torch-0.4.0-cp36-cp36m-linux_x86_64.whl'
with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [98]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = 'score.py', 
                                                 runtime = 'python',
                                                 conda_file = 'myenv.yml',
                                                 docker_file = 'docker_steps',
                                                 dependencies = ['./'],
                                                 description = "Chest x-ray image classification for diseases",
                                                 tags = {'area': "health", 'type': "classification"})

image = ContainerImage.create(name = "xray-image",
                             models = [model], 
                             image_config = image_config, 
                             workspace = ws)

image.wait_for_creation(show_output = True)

Creating image
Running.........................................................................................................................................................................................................................................
SucceededImage creation operation finished for image xray-image:6, operation "Succeeded"


## Attach AKS Cluster

In [27]:
from azureml.core.compute import AksCompute, ComputeTarget

On the first run of the notebook, provision a new AKS cluster.

In [52]:
# aks_cluster_name = "aks-chest-xray"
# prov_config = AksCompute.provisioning_configuration()

# aks_target = ComputeTarget.create(workspace = ws, 
#                                       name = aks_cluster_name, 
#                                       provisioning_configuration = prov_config)

In [53]:
# aks_target.wait_for_completion(show_output = True)
# print(aks_target.provisioning_state)
# print(aks_target.provisioning_errors)

Creating...........................................................................................
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
None


On subsequent runs, reference existing AKS cluster.

In [28]:
resource_id = "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/ghiordanchestxray03rsg/providers/Microsoft.ContainerService/managedClusters/aks-chest-xray379972399"
aks_cluster_name = "aks-chest-xray"

attach_config = AksCompute.attach_configuration(resource_id=resource_id)
aks_target = ComputeTarget.attach(workspace=ws, name=aks_cluster_name, attach_configuration=attach_config)
# Wait for the operation to complete
aks_target.wait_for_completion(True)

SucceededProvisioning operation finished, operation "Succeeded"


## Deploy web service to AKS

In [29]:
from azureml.core.webservice import Webservice, AksWebservice

In [30]:
#Set the web service configuration (using default here)
aks_config = AksWebservice.deploy_configuration()

In [99]:
%%time
aks_service_name ='aks-service-xray-6'

aks_service = Webservice.deploy_from_image(workspace = ws, 
                                           name = aks_service_name,
                                           image = image,
                                           deployment_config = aks_config,
                                           deployment_target = aks_target)
aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

Creating service
Running.........................................
SucceededAKS service creation operation finished, operation "Succeeded"
Healthy
CPU times: user 809 ms, sys: 32.5 ms, total: 841 ms
Wall time: 4min 16s


In [55]:
# print(ws.webservices['aks-service-xray-6'].get_logs())

## Test web service

In [101]:
%%time

import json
import cv2
import pickle, base64
import pandas as pd
import numpy as np

#parameters
image_dir = "./../../../data/chestxray/ChestX-ray8/ChestXray-NIHCC/images"
image_name = "00000003_000.png"
resized_height = 224
resized_width = 224

def as_string_b64encoded_pickled(input_object):
     #b64encode returns bytes class, make it string by calling .decode('utf-8')
    return (base64.b64encode(pickle.dumps(input_object))).decode('utf-8')

full_path = os.path.join(image_dir, image_name)

cv2_image = cv2.resize(cv2.imread(full_path), (resized_height, resized_width))
# cv2_images = np.expand_dims(cv2_image, 0)                                               
test_images = json.dumps({"data": cv2_image.tolist()})
test_images = bytes(test_images, encoding = "utf8")
# encoded_images = as_string_b64encoded_pickled(cv2_images)
# raw_data = pd.DataFrame(data=[[encoded_image]], columns=["encoded_image"])

output_dict = json.loads(aks_service.run(test_images))
prediction = output_dict["chestXrayScore"]
print(prediction)

[1.30585119e-01 1.03531078e-04 1.65205672e-02 1.18964024e-01
 3.62095051e-02 6.71617389e-02 7.08098011e-03 1.68860499e-02
 1.08668078e-02 1.77454363e-04 2.34917458e-02 3.49089913e-02
 1.68611444e-11 5.03102923e-03]
CPU times: user 58 ms, sys: 4.08 ms, total: 62.1 ms
Wall time: 2.69 s


## Clean up

In [ ]:
# %%time
# aks_service.delete()
# image.delete()
# model.delete()